In [2]:
import pandas as pd
def filter_df(df, prefix=""):
    return df[(df['Origin'] != df[f'{prefix}choice']) | (df['Origin'] != df[f'{prefix}flipped_choice'])]

In [3]:
df = pd.read_csv('data/failed_distinguishes_for_llama3.1-70B-long.csv')
df['total'] = 1
print(len(df.groupby(by=['w_str', 'm_str', 'Origin', 'attack1_id', 'attack2_id']).agg({'total': 'sum'})))
df.groupby(by=['m_str', 'w_str', 'Origin', 'attack1_id', 'attack2_id']).agg({'total': 'sum'})


13


total
m_str           w_str    Origin attack1_id attack2_id       
SentenceMutator Adaptive A      3          4              10
                                           5              10
                KGW      A      1          2               5
                                3          4               9
                         B      0          1               8
                SIR      A      0          2              16
                                1          2               5
                                3          5              17
                                4          5              37
                         B      0          1               5
                                           2               5
                                3          4              36
                                           5              24

In [7]:
df = pd.read_csv('results/redemption_long_with_o3-mini.csv')
# filter df to include rows where Origin = 2nd_choice
df = filter_df(df, prefix="3rd_")
print(len(df), list(df.index))
df.to_csv('data/failed_redemption_long_with_o3-mini.csv')
df.drop(columns=['choice', 'flipped_choice', '3rd_choice', '3rd_flipped_choice', 'Origin'], inplace=True)

48 [20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 47, 48, 51, 52, 53, 54, 99, 100, 103, 106, 107, 108, 109, 111, 115, 117, 120, 123, 126, 128, 129, 131, 133, 139, 147, 162]


In [8]:
df

,P,Num,prompt,o_str,w_str,m_str,prompt_type,entropy,attack1_id,attack2_id,group_id,origin_A,origin_B
20,Exploring space is crucial for expanding our u...,72,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
21,Exploring space is crucial for expanding our u...,84,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
22,Exploring space is crucial for expanding our u...,96,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
23,Exploring space is crucial for expanding our u...,108,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
24,Exploring space is crucial for expanding our u...,120,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
25,Exploring space is crucial for expanding our u...,132,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
27,Venturing into space is crucial for deepening ...,156,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
28,Venturing into space is crucial for deepening ...,168,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
29,Exploring space is essential for expanding our...,180,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...
30,Venturing into space is crucial for deepening ...,192,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,4,2,Space exploration is often perceived as an ext...,Space exploration is essential for the advance...


In [12]:
row = df.iloc[0]
P = row['P']
origin_A = row['origin_A']
origin_B = row['origin_B']

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: Since the emergence of humanity, an unquenchable desire for understanding has propelled individuals to venture into the vast expanse of space, fueled by an unyielding pursuit to unravel the mysteries of existence, encompassing the possibility of extraterrestrial life and the discovery of unconventional sustenance sources. Each pioneering space mission, like the historic Apollo 11 moon landing, has greatly broadened our comprehension of the universe, catalyzing the creation of innovative technologies that profoundly impact our daily routines on Earth. The development of solar panels, which convert sunlight into electrical energy, was motivated by the need for dependable power systems in space missions. The proliferation of telecommunications has connected the globe, enabling effortless communication in various contexts. Unraveling the universe's secrets is pivotal in addressing two interrelated dilemmas: preserving the planet's ecological balance and exploiting its resour